In [ ]:
#!conda install -y pandas 

In [ ]:
#!conda install -y joblib

In [1]:
#%matplotlib inline
#%config InlineBackend.figure_format = 'retina'
import sys, platform, os
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy import integrate
from scipy import optimize
from scipy import interpolate
from scipy import special

# Parallelizing
import multiprocessing as mp
import numba
from numba import njit, prange
import joblib
from joblib import Parallel, delayed
#----------------------------------------------

from mpl_toolkits import mplot3d
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable 
%pylab inline 
%config InlineBackend.figure_format = 'svg' 
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams['mathtext.fontset'] = 'cm'
#Assume installed from github using "git clone --recursive https://github.com/cmbant/CAMB.git"
#This file is then in the docs folders
#camb_path = os.path.realpath(os.path.join(os.getcwd(),'..'))
#sys.path.insert(0,camb_path)
import camb
from camb import model, initialpower
print('Using CAMB %s installed at %s'%(camb.__version__,os.path.dirname(camb.__file__)))

Populating the interactive namespace from numpy and matplotlib
Using CAMB 1.1.1 installed at /home/alba/anaconda2/envs/ipykernel_py3/lib/python3.8/site-packages/camb


In [2]:
# Load file with transfers
cmb_transfer_CAMB = pd.read_csv('./Files/cmb_transfer_TT_EE.txt', sep=" ", header=None)
cmb_transfer_CAMB.columns=['ell','k','dk','gT','gP']

MemoryError: Unable to allocate 3.03 GiB for an array with shape (5, 81228751) and data type float64

In [ ]:
# Selectes the values of ell in the dataframe
ell_values_CAMB = cmb_transfer_CAMB['ell'].unique()

In [ ]:
ell_values_CAMB

In [ ]:
# Compute r_sampling according to Table 1 of Liguori et al. PRD, 76, 105016 (2007)
tau0 = 14142.0762
r = tau0 + 500.0 
r_count = 0.0
r_values = [r] #starting from r

while r > r_sample:  
    r_count+=1.0
    if r_count <= 450.0:
        r_sample = 3.5 #sample densely during recombination
    elif r_count <= 485.0:
        r_sample = 105.0
    elif r_count <= 515.0:
        r_sample = 10.0 #sample densely during reionization
    else:
        r_sample = 105.0
    r-=r_sample
    r_values.append(r)

$$\beta (\ell, r) = 4\pi \mathcal{A}_\mathrm{s}\int \frac{\mathrm{d}k}{k}[g_{T\ell}(k)j_\ell(kr)] \left(\frac{k}{k_\star}\right)^{n_\mathrm{s}-1}$$

In [ ]:
def compute_beta(ell,
                 values_of_r,
                 transfer_function, 
                 source):
    
    A_s = 2.1056e-9
    n_s = 0.9665
    k_pivot = 0.05
    
    reduced_df = transfer_function[transfer_function['ell'] == ell]
    array_beta_r = np.zeros(len(values_of_r),'float64')
        
    for index_r in range(len(values_of_r)):
        array_beta_r[index_r] = 4.0 * np.pi * A_s  * integrate.trapz(reduced_df[source].values*
                                                special.spherical_jn(int(ell),reduced_df['k'].values*
                                                                     values_of_r[index_r])/
                                                reduced_df['k'].values*
                                                np.power(reduced_df['k'].values/k_pivot, n_s-1),
                                                x = reduced_df['k'].values)
        
    return array_beta_r

In [ ]:
# Number of cpu available for parallelizing
n_cpu = 1

In [28]:
beta_function_temp = Parallel(n_jobs=n_cpu)(delayed(compute_beta)(ell,r_values,cmb_transfer_CAMB, 'gT') 
                                            for ell in ell_values_CAMB.astype(int))

CPU times: user 563 ms, sys: 1.13 s, total: 1.7 s
Wall time: 20 s


In [28]:
beta_5000_temp = compute_beta(5000,r_values,cmb_transfer_CAMB, 'gT') 

CPU times: user 563 ms, sys: 1.13 s, total: 1.7 s
Wall time: 20 s


In [18]:
beta_function_pol = Parallel(n_jobs=n_cpu)(delayed(compute_beta)(ell,r_values,cmb_transfer_CAMB, 'gP') 
                                           for ell in ell_values_CAMB.astype(int))

CPU times: user 28.6 s, sys: 6.15 ms, total: 28.6 s
Wall time: 28.6 s


In [ ]:
np.savetxt('beta_function_lmax5000_T.txt',beta_function_temp)

In [ ]:
np.savetxt('beta_function_lmax5000_P.txt',beta_function_pol)